<a href="https://colab.research.google.com/github/SURAJ28092001/army_bot/blob/main/Army_Bot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import json 
import numpy as np 
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, GlobalAveragePooling1D
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder

In [ ]:
with open('Army_Intents.json') as intents:
    data = json.load(intents)

training_sentences = []
training_labels = []
label = []
responses = []

for i in data['intents']:
    for j in i['patterns']:
        training_sentences.append(j)
        training_labels.append(i['tag'])
    responses.append(i['responses'])

    if i['tag'] not in label:
      label.append(i['tag'])

num_classes = len(label)


In [ ]:
lbl_encoder = LabelEncoder()
lbl_encoder.fit(training_labels)
training_labels = lbl_encoder.transform(training_labels)

In [ ]:
tokenizer = Tokenizer(num_words=1000,oov_token="<OOV>")
tokenizer.fit_on_texts(training_sentences)
word_index = tokenizer.word_index
seq = tokenizer.texts_to_sequences(training_sentences)
pad_seq = pad_sequences(seq,truncating='post',maxlen=20)

In [ ]:
model = Sequential()
model.add(Embedding(1000, 16, input_length=20))
model.add(GlobalAveragePooling1D())
model.add(Dense(16, activation='relu'))
model.add(Dense(16, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss='sparse_categorical_crossentropy', 
              optimizer='adam', metrics=['accuracy'])

model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, 20, 16)            16000     
                                                                 
 global_average_pooling1d_2   (None, 16)               0         
 (GlobalAveragePooling1D)                                        
                                                                 
 dense_6 (Dense)             (None, 16)                272       
                                                                 
 dense_7 (Dense)             (None, 16)                272       
                                                                 
 dense_8 (Dense)             (None, 17)                289       
                                                                 
Total params: 16,833
Trainable params: 16,833
Non-trainable params: 0
__________________________________________________

In [ ]:
epochs = 500
history = model.fit(pad_seq,np.array(training_labels),epochs=epochs)

Epoch 1/500
3/3 [==============================] - 0s 4ms/step - loss: 3.2549e-04 - accuracy: 1.0000
Epoch 2/500
3/3 [==============================] - 0s 5ms/step - loss: 3.2485e-04 - accuracy: 1.0000
Epoch 3/500
3/3 [==============================] - 0s 6ms/step - loss: 3.2398e-04 - accuracy: 1.0000
Epoch 4/500
3/3 [==============================] - 0s 5ms/step - loss: 3.2333e-04 - accuracy: 1.0000
Epoch 5/500
3/3 [==============================] - 0s 6ms/step - loss: 3.2257e-04 - accuracy: 1.0000
Epoch 6/500
3/3 [==============================] - 0s 4ms/step - loss: 3.2201e-04 - accuracy: 1.0000
Epoch 7/500
3/3 [==============================] - 0s 5ms/step - loss: 3.2120e-04 - accuracy: 1.0000
Epoch 8/500
3/3 [==============================] - 0s 5ms/step - loss: 3.2079e-04 - accuracy: 1.0000
Epoch 9/500
3/3 [==============================] - 0s 5ms/step - loss: 3.2019e-04 - accuracy: 1.0000
Epoch 10/500
3/3 [==============================] - 0s 6ms/step - loss: 3.1937e-04 - accura

In [ ]:
model.save('chat_model')

In [ ]:
import pickle

# to save the fitted tokenizer
with open('tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)
    
# to save the fitted label encoder
with open('label_encoder.pickle', 'wb') as ecn_file:
    pickle.dump(lbl_encoder, ecn_file, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
with open("Army_Intents.json") as file:
    data = json.load(file)
model = keras.models.load_model('chat_model')
# load tokenizer object
with open('tokenizer.pickle', 'rb') as handle:
    tokenizer = pickle.load(handle)

# load label encoder object
with open('label_encoder.pickle', 'rb') as enc:
    lbl_encoder = pickle.load(enc)

print(lbl_encoder)


LabelEncoder()


In [ ]:
while True:
  print("User : ",end="")
  inp = input()
  if inp.lower() == 'quit':
    break
  result = model.predict(keras.preprocessing.sequence.pad_sequences(tokenizer.texts_to_sequences([inp]),truncating='post',maxlen=20))
  #print(result)
  tag = lbl_encoder.inverse_transform([np.argmax(result)])
  for i in data['intents']:
    if tag == i['tag']:
      print("ChatBot : ",np.random.choice(i['responses']))


User : Hi
ChatBot :  Hello
User : What is your name ?
ChatBot :  I'm Agniveer!
User : Agniveer, what is Indian Army ?
ChatBot :  The Indian Army is the land-based branch and the largest component of the Indian Armed Forces. The President of India is the Supreme Commander of the Indian Army, and its professional head is the Chief of Army Staff, who is a four-star general.
User : So, who is the Chief of Army Staff ?
ChatBot :  The current Chief of Army Staff is Gen. Manoj Pande.
User : And who is the current CDS ?
ChatBot :  The current Chief of Defence Staff is Gen Anil Chouhan
User : Great
ChatBot :  Happy to help!
User : How many commands are there in Indian Army
ChatBot :  There are 7 Army Commands in India viz. Northern Command, Southern Command, Eastern Command, Western Command, Central Command, South Western Command & Training Command
User : Tell me about northern command
ChatBot :  The Northern Command is located at Udhampur
User : And Southern command
ChatBot :  It is an army co